In [85]:
import os
import time
from collections import defaultdict
from itertools import combinations, product
from collections import defaultdict
import numpy as np
from queue import LifoQueue
import itertools

import pandas as pd
from gensim.models import Word2Vec

In [86]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [87]:
def read_data():
    DATA_PATH = r'parus_results'
    files = os.listdir(DATA_PATH)
    trees_df = pd.DataFrame(columns=['id', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel'])

    for file in files:
        full_dir = os.path.join(DATA_PATH, file)
        name = file.split('.')[0]
        with open(full_dir, encoding='utf-8') as f:
            this_df = pd.read_csv(f, sep='\t',
                                  names=['id', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel'])
            if this_df['id'].duplicated().any():
                start_of_subtree_df = list(this_df.groupby(this_df.id).get_group(1).index)
                boundaries = start_of_subtree_df + [max(list(this_df.index)) + 1]
                list_of_dfs = [this_df.iloc[boundaries[n]:boundaries[n + 1]] for n in range(len(boundaries) - 1)]
                local_counter = 1
                for df in list_of_dfs:
                    df['sent_name'] = name + '_' + str(local_counter)
                    trees_df = pd.concat([trees_df, df], ignore_index=True)
                    local_counter += 1
            else:
                this_df['sent_name'] = name
                trees_df = pd.concat([trees_df, this_df], ignore_index=True)

    # delete useless data
    trees_df = trees_df.drop(columns=['upostag', 'xpostag', 'feats'], axis=1)
    trees_df.drop(index=[11067], inplace=True)
    trees_df.loc[13742, 'deprel'] = 'разъяснит'

    # delete relations of type PUNC and reindex
    trees_df_filtered = trees_df[trees_df.deprel != 'PUNC']
    trees_df_filtered = trees_df_filtered.reset_index(drop=True)
    trees_df_filtered.index = trees_df_filtered.index + 1
    return trees_df_filtered

In [88]:
def train_word2vec(trees_df_filtered, lemmas):
    lemma_sent_df = trees_df_filtered[['lemma', 'sent_name']]
    lemma_sent_dict = {}
    for name, group in lemma_sent_df.groupby('sent_name'):
        lemma_sent_dict[name] = []
        for _, row in group.iterrows():
            lemma_sent_dict[name].append(row['lemma'])
    model = Word2Vec(list(lemma_sent_dict.values()), min_count=1)
    similar_dict = {}
    for lemma in lemmas:
        similar_dict[lemma] = model.most_similar(lemma)

In [89]:
class Node:
    def __init__(self, id, lemma=None, form=None, sent_name=None, is_included=False):
        self.id = id
        self.lemma = lemma
        self.form = form
        self.sent_name = sent_name
        self.is_included = is_included


class Edge:
    def __init__(self, node_id_from, node_id_to, weight):
        self.node_from = node_id_from
        self.node_to = node_id_to
        self.weight = weight  # relation type

In [90]:
class Tree:

    def __init__(self):
        self.edges = []
        self.nodes = []
        # self.heights = []
        self.inactive = set()
        self.created = set()
        self.heights = {}
        self.edges_dict_from = {}
        self.edges_dict_to = {}
        self.nodes_dict_id = {}

    def set_help_dict(self):
        self.edges_dict_from = {k: list(v) for k, v in itertools.groupby(sorted(self.edges, key=lambda x: x.node_from),
                                                                         key=lambda x: x.node_from)}
        self.nodes_dict_id = {node.id: node for node in self.nodes}
        self.edges_dict_to = {k: list(v) for k, v in itertools.groupby(self.edges, key=lambda x: x.node_to)}

    def add_node(self, node):
        self.nodes.append(node)

    def add_edge(self, edge):
        self.edges.append(edge)

    def get_node(self, node_id):
        return self.nodes_dict_id.get(node_id)  # return Node class instance

    def get_children(self, node_id):
        edges = self.edges_dict_from.get(node_id)
        # only_active = list(filter(lambda edge: edge.node_to not in self.inactive and edge.node_from not in self.inactive, edges if edges is not None else []))
        # return list(map(lambda x: x.node_to, only_active if only_active is not None else []))
        return set(map(lambda x: x.node_to, edges if edges is not None else []))

    def get_edge(self, to_id):
        return self.edges_dict_to.get(to_id)

    def remove_edge(self, to_id):
        self.edges = list(filter(lambda x: x.node_to != to_id, self.edges))

    def copy_node_details(self, existing_node):
        new_node = Node(id=len(self.nodes),
                    form=existing_node.form,
                    sent_name=existing_node.sent_name,
                    is_included=existing_node.is_included)
        self.nodes_dict_id[new_node.id] = new_node
        self.created.add(new_node.id)
        return new_node

    def add_new_edges(self, new_node_id, children):
        for child_id in children:
            new_edge = Edge(new_node_id, child_id, self.get_edge(child_id)[0].weight)
            if child_id in self.edges_dict_to.keys():
                self.edges_dict_to[child_id].append(new_edge)
            else:
                self.edges_dict_to[child_id] = [new_edge]
            self.edges.append(new_edge)
            if new_node_id in self.edges_dict_from.keys():
                self.edges_dict_from[new_node_id].append(new_edge)
            else:
                self.edges_dict_from[new_node_id] = [new_edge]

    def add_edge_to_dict(self, edge):
        self.edges_dict_to[edge.node_to] = [edge]
        if edge.node_from in self.edges_dict_from.keys():
            self.edges_dict_from[edge.node_from].append(edge)
        else:
            self.edges_dict_from[edge.node_from] = edge
        self.edges.append(edge)

    def add_node_to_dict(self, node):
        self.nodes_dict_id[node.id] = node
        self.nodes.append(node)

    def add_inactive(self, node_id):
        self.inactive.add(node_id)
        
    def calculate_heights(self):
        visited = np.full(len(self.nodes), False, dtype=bool)
        # self.heights = np.full(len(self.nodes), -1, dtype=int)  # all heights are -1 initially
        stack = LifoQueue()
        stack.put(0)
        prev = None
        while stack.qsize() > 0:
            curr = stack.get()
            stack.put(curr)
            if not visited[curr]:
                visited[curr] = True
            children = self.get_children(curr)
            if len(children) == 0:
                self.heights[curr] = [0]
                prev = curr
                stack.get()
            else:
                all_visited_flag = True
                for child in children:
                    if not visited[child]:
                        all_visited_flag = False
                        stack.put(child)
                if all_visited_flag:
                    curr_height = []
                    if len(children) > 1:
                        for child in children:
                            for child_height in self.heights[child]:
                                curr_height.append(child_height + 1)
                    else:
                        curr_height = [h + 1 for h in self.heights[prev]]
                    self.heights[curr] = list(set(curr_height))
                    prev = curr
                    stack.get()

In [91]:
def construct_tree(trees_df_filtered, dict_lemmas, dict_rel):
    # construct a tree with a list of edges and a list of nodes
    whole_tree = Tree()
    root_node = Node(0, 0)  # add root
    Tree.add_node(whole_tree, root_node)
    for name, group in trees_df_filtered.groupby('sent_name'):
        row_ids = trees_df_filtered.index[trees_df_filtered.sent_name == name].to_list()
        # temporary dictionary for remapping indices
        temp_dict = {key: row_ids[ind] for ind, key in enumerate(group.id.to_list())}
        temp_dict[0] = 0
        for _, row in group.iterrows():
            new_id = temp_dict.get(row['id'])
            new_node = Node(new_id, dict_lemmas.get(row['lemma']), row['form'], row['sent_name'])
            Tree.add_node(whole_tree, new_node)
            new_edge = Edge(temp_dict.get(row['head']), new_id, dict_rel.get(row['deprel']))
            Tree.add_edge(whole_tree, new_edge)
    return whole_tree

In [92]:
def add_children_to_parents(k_2, filtered_groups, whole_tree, curr_height, old_node_new_nodes):
    all_parents = set()
    for k, v in filtered_groups.items():
        for v_id in list(v):
            edge_to_curr = Tree.get_edge(whole_tree, v_id)
            Tree.get_node(whole_tree, v_id).is_included = True
            if edge_to_curr is not None:
                parent = edge_to_curr[0].node_from
                if max(whole_tree.heights[parent]) > curr_height:
                    all_parents.add(parent)
                if v_id in old_node_new_nodes.keys():
                    lemmas_to_visit = old_node_new_nodes[v_id]
                else:
                    lemmas_to_visit = [k]
                for lemma in lemmas_to_visit:
                    label_for_child = str(edge_to_curr[0].weight) + str(lemma)
                    if parent not in k_2.keys():
                        k_2[parent] = {label_for_child}
                    else:
                        k_2[parent].add(label_for_child)
    return all_parents

In [93]:
def add_additional_children_to_parents(k_2, whole_tree, all_parents):
    additional_child_nodes = {}
    for parent in all_parents:
        for child_id in Tree.get_children(whole_tree, parent):
            edge_to_curr = Tree.get_edge(whole_tree, child_id)[0]
            child_node = Tree.get_node(whole_tree, child_id)
            if not child_node.is_included:
                label_for_child = str(edge_to_curr.weight) + str(child_node.lemma)
                k_2[parent].add(label_for_child)
                child_node.is_included = True
                if parent not in additional_child_nodes.keys():
                    additional_child_nodes[label_for_child] = child_id
                else:
                    additional_child_nodes[label_for_child].append(child_id)
    return additional_child_nodes

In [94]:
def produce_combinations(k_2, v_id, str_sequence_help, str_sequence_help_reversed, equal_nodes, equal_nodes_mapping):
    if len(equal_nodes) > 0:
        list_for_combinations = []
        prepared_k_2 = set()
        included_labels = []
        for child_tree in k_2[v_id]:
            if child_tree in [item for sublist in list(equal_nodes.values()) for item in sublist] or child_tree in equal_nodes.keys():
                if child_tree in equal_nodes_mapping.keys():
                    actual_label = equal_nodes_mapping[child_tree]
                else:
                    actual_label = child_tree
                if actual_label not in included_labels:
                    list_for_combinations.append(equal_nodes[actual_label])
                    included_labels.append(actual_label)
            else:
                prepared_k_2.add(child_tree)
        combinations_repeated = list(product(*(list_for_combinations)))
        # test1 = ['10', '11', '12', '13']
        # test2 = ['14', '15']
        # combinations_repeated = list(product(*([list_for_combinations[0], test1, test2])))
        all_combinations = []
        if len(prepared_k_2) > 0:
            for l in combinations_repeated:
                merged = list(l) + list(prepared_k_2)
                all_combinations.extend(list(combinations(merged, i)) for i in range(1, len(merged) + 1))
        else:
            for l in combinations_repeated:
                all_combinations.extend(list(combinations(list(l), i)) for i in range(1, len(list(l)) + 1))
    else:
        list_for_combinations = k_2[v_id]
        all_combinations = [list(combinations(list_for_combinations, i)) for i in range(1, len(list_for_combinations) + 1)]
    all_combinations_str_joined = set()
    for comb in all_combinations:
        for tup in comb:
            combs = [str(item) for item in sorted(list(map(int, list(tup))))]
            joined_label = ''.join(sorted(combs))
            if joined_label not in str_sequence_help.keys():
                str_sequence_help[joined_label] = combs
                str_sequence_help_reversed[tuple(combs)] = joined_label
            all_combinations_str_joined.add(joined_label)
    return all_combinations_str_joined

In [95]:
def get_nodeid_repeats(filtered_combination_ids, str_sequence_help):
    dict_nodeid_comb = {}
    for k, v in filtered_combination_ids.items():
        for v_i in v:
            if v_i in dict_nodeid_comb.keys():
                dict_nodeid_comb[v_i].append(str_sequence_help.get(k))
            else:
                dict_nodeid_comb[v_i] = [str_sequence_help.get(k)]
    return dict_nodeid_comb

In [96]:
def compute_part_new_new(whole_tree, lemma_count, grouped_heights):
    classes_subtreeid_nodes = {}
    classes_subtreeid_nodes_list = {}
    unique_subtrees_mapped_global_subtree_lemma = {}
    old_node_new_nodes = {}
    equal_nodes_mapping = {}
    k_2 = {}  # identifiers of edges of subtrees
    lemma_nodeid_dict = {}
    for nodes in grouped_heights:
        curr_height = nodes[0]
        print(curr_height)
        start = time.time()
        id_lemma_dict = {node.id: node.lemma for node in nodes[1]}
        grouped_lemmas = defaultdict(list)
        for key, value in id_lemma_dict.items():
            grouped_lemmas[value].append(key)
        all_parents = add_children_to_parents(k_2, grouped_lemmas, whole_tree, curr_height, old_node_new_nodes)
        additional_child_nodes = add_additional_children_to_parents(k_2, whole_tree, all_parents)
        for additional_child, child_id in additional_child_nodes.items():
            if additional_child not in lemma_nodeid_dict.keys():
                lemma_nodeid_dict[additional_child] = {child_id}
            else:
                lemma_nodeid_dict[additional_child].add(child_id)
        for lemma, ids in grouped_lemmas.items():
            for v_id in ids:
                edge_to_curr = Tree.get_edge(whole_tree, v_id)
                if edge_to_curr is not None:
                    label_for_child = str(edge_to_curr[0].weight) + str(lemma)
                    if label_for_child not in lemma_nodeid_dict.keys():
                        lemma_nodeid_dict[label_for_child] = {v_id}
                    else:
                        lemma_nodeid_dict[label_for_child].add(v_id)
        filtered_groups = {k: v for k, v in grouped_lemmas.items() if len(v) > 1}

        if curr_height != 0:  # not applicable to leaves, leaves don't have subtrees
            for lemma, ids in filtered_groups.items():
                combination_ids = {}
                str_sequence_help = {}
                str_sequence_help_reversed = {}

                # generate combinations
                for v_id in ids:
                    equal_nodes = {}
                    # only for duplicating nodes
                    children = Tree.get_children(whole_tree, v_id) - whole_tree.created
                    # children = list(filter(lambda child: child not in whole_tree.created, Tree.get_children(whole_tree, v_id)))
                    for child in children:
                        if child in old_node_new_nodes.keys():
                            edge_to_child = Tree.get_edge(whole_tree, child)[0]
                            child_node = Tree.get_node(whole_tree, child)
                            w = str(edge_to_child.weight)
                            actual_label = w + str(child_node.lemma)
                            if actual_label not in equal_nodes.keys():
                                merge = []
                                for l in old_node_new_nodes[child]:
                                    new_label = w + str(l)
                                    merge.append(new_label)
                                    equal_nodes_mapping[new_label] = actual_label
                                equal_nodes[actual_label] = merge
                            else:
                                merge = []
                                for l in old_node_new_nodes[child]:
                                    new_label = w + str(l)
                                    merge.append(new_label)
                                    equal_nodes_mapping[new_label] = actual_label
                                equal_nodes[actual_label].extend(merge)
                    all_combinations_str_joined = produce_combinations(k_2, v_id, str_sequence_help,
                                                                       str_sequence_help_reversed, equal_nodes,
                                                                       equal_nodes_mapping)
                    for label in all_combinations_str_joined:
                        if label in combination_ids.keys():
                            combination_ids[label].append(v_id)
                        else:
                            combination_ids[label] = [v_id]

                filtered_combination_ids = {k: v for k, v in combination_ids.items() if len(v) > 1}
                for tree_label, node_list in filtered_combination_ids.items():
                    if tree_label not in unique_subtrees_mapped_global_subtree_lemma.keys():
                        unique_subtrees_mapped_global_subtree_lemma[tree_label] = lemma_count
                        lemma_count += 1
                # 16: [['107', '919'], ['208', '919'], ['919'], ['107'], ['208']]
                dict_nodeid_comb = get_nodeid_repeats(filtered_combination_ids, str_sequence_help)
                for node_id, node_subtrees in dict_nodeid_comb.items():
                    existing_node = Tree.get_node(whole_tree, node_id)
                    edge_to_curr = Tree.get_edge(whole_tree, node_id)[0]
                    children = Tree.get_children(whole_tree, node_id)
                    for subtree in node_subtrees:
                        subtree_text = str_sequence_help_reversed.get(tuple(subtree))
                        subtree_new_label = unique_subtrees_mapped_global_subtree_lemma.get(subtree_text)

                        # add new node with a new lemma
                        new_node = Tree.copy_node_details(whole_tree, existing_node)
                        new_node.lemma = subtree_new_label
                        Tree.add_node_to_dict(whole_tree, new_node)

                        # add new node to node aliases
                        if node_id not in old_node_new_nodes.keys():
                            old_node_new_nodes[node_id] = [new_node.lemma]
                        else:
                            old_node_new_nodes[node_id].append(new_node.lemma)

                        # add an edge to it
                        edge = Edge(edge_to_curr.node_from, new_node.id, edge_to_curr.weight)
                        Tree.add_edge_to_dict(whole_tree, edge)

                        # add new node to parent
                        parent_subtree_text = str(edge_to_curr.weight) + str(subtree_new_label)
                        if parent_subtree_text not in lemma_nodeid_dict.keys():
                            lemma_nodeid_dict[parent_subtree_text] = {new_node.id}
                        else:
                            lemma_nodeid_dict[parent_subtree_text].add(new_node.id)

                        subtree_children = []
                        children_nodes = {}
                        for child in children:
                            child_node = Tree.get_node(whole_tree, child)
                            str_lemma = str(child_node.lemma)
                            child_node_lemma = str(Tree.get_edge(whole_tree, child)[0].weight) + str_lemma
                            children_nodes[child_node_lemma] = child_node.id
                            children_nodes[str_lemma] = child_node.id
                        for subtree_node in subtree:
                            if subtree_node not in lemma_nodeid_dict.keys():
                                if subtree_node in equal_nodes_mapping.keys():
                                    target_child = list(set(lemma_nodeid_dict[equal_nodes_mapping[subtree_node]]) & children)[0]
                                else:
                                    target_child = children_nodes[subtree_node]
                            else:
                                intersection = set(lemma_nodeid_dict[subtree_node]) & children
                                if len(intersection) == 0:
                                    target_child = children_nodes[subtree_node]
                                else:
                                    target_child = list(intersection)[0]
                            subtree_children.append(target_child)

                        if len(subtree_children) > 0:
                            # add edges to subtree's children from new node
                            Tree.add_new_edges(whole_tree, new_node.id, subtree_children)

                            # assign class
                            if subtree_new_label not in classes_subtreeid_nodes.keys():
                                classes_subtreeid_nodes[subtree_new_label] = [new_node.id]
                            else:
                                classes_subtreeid_nodes[subtree_new_label].append(new_node.id)

                            subtree_deep_children = set()
                            for subtree_lemma in list(map(lambda x: Tree.get_node(whole_tree, x).lemma, subtree_children)):
                                if subtree_lemma in classes_subtreeid_nodes_list.keys():
                                    subtree_deep_children.update(classes_subtreeid_nodes_list[subtree_lemma])
                            subtree_deep_children.update(subtree_children)
                            only_active = subtree_deep_children - whole_tree.inactive
                            # only_active = (whole_tree.inactive.symmetric_difference(set_children))&set_children

                            if subtree_new_label not in classes_subtreeid_nodes_list.keys():
                                classes_subtreeid_nodes_list[subtree_new_label] = only_active
                            else:
                                classes_subtreeid_nodes_list[subtree_new_label].update(only_active)
                            classes_subtreeid_nodes_list[subtree_new_label].add(new_node.id)

                    # remove old node and edges to/from it
                    Tree.add_inactive(whole_tree, node_id)
        print(time.time() - start)
    return classes_subtreeid_nodes

In [97]:
trees_df_filtered = read_data()
    # # TEST - тест на первых 3х предложениях
trees_df_filtered = trees_df_filtered.head(5015)  # 341 - all? 48 - 3 # 3884 # 5015
    # # trees_df_filtered = trees_df_filtered[trees_df_filtered.sent_name == '48554_5']
    #
    # get all lemmas and create a dictionary to map to numbers
dict_lemmas = {lemma: index for index, lemma in enumerate(dict.fromkeys(trees_df_filtered['lemma'].to_list()), 1)}
    # get all relations and create a dictionary to map to numbers
dict_rel = {rel: index for index, rel in enumerate(dict.fromkeys(trees_df_filtered['deprel'].to_list()))}
train_word2vec(trees_df_filtered, dict_lemmas)
    #
start = time.time()
whole_tree = construct_tree(trees_df_filtered, dict_lemmas, dict_rel)
print('Time on constructing the tree: ' + str(time.time() - start))
    # whole_tree = new_test()
Tree.set_help_dict(whole_tree)
    # partition nodes by height
start = time.time()
Tree.calculate_heights(whole_tree)
print('Time on calculating all heights: ' + str(time.time() - start))

heights_dictionary = {Tree.get_node(whole_tree, node_id): heights for node_id, heights in
                          whole_tree.heights.items()}
grouped_heights = defaultdict(list)
for node, heights in heights_dictionary.items():
    for height in heights:
        grouped_heights[height].append(node)
grouped_heights = sorted(grouped_heights.items(), key=lambda x: x[0])
    
dict_lemmas_size = max(set(map(lambda x: x.lemma, whole_tree.nodes)))

    # classes for partial repeats
    
#     classes_part = compute_part_new_new(whole_tree, dict_lemmas_size, grouped_heights)

<ipython-input-87-5441d0faa27a>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sent_name'] = name + '_' + str(local_counter)
<ipython-input-88-7ea0cf5d05b9>:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  similar_dict[lemma] = model.most_similar(lemma)


Time on constructing the tree: 1.0034382343292236
Time on calculating all heights: 0.15526604652404785


In [98]:
start = time.time()
%lprun -f compute_part_new_new compute_part_new_new(whole_tree, dict_lemmas_size, grouped_heights)
print('Time on calculating partial repeats: ' + str((time.time() - start) % 60))

0
0.18599987030029297
1
1.7412240505218506
2
3.5044620037078857
3
10.251235961914062
4
16.02889084815979
5
16.064427852630615
6
0.9864029884338379
7
0.25371289253234863
8
0.3043060302734375
9
0.09629392623901367
10
0.08630776405334473
11
0.000370025634765625
12
0.0008518695831298828
13
0.09117484092712402
14
0.06423592567443848
15
0.00015783309936523438
Time on calculating partial repeats: 49.70806097984314


In [70]:
149     15739    2561829.0    162.8      6.9   

SyntaxError: invalid syntax (<ipython-input-70-69a826bf863d>, line 1)